# 필요한 라이브러리 import

In [1]:
import csv
import pandas as pd
import os
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import warnings
import pickle
plt.rc('font',family='Malgun Gothic')
plt.rc('font',size=10)
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import load_model
import tensorflow as tf

warnings.filterwarnings(action='ignore')

In [2]:
with open('./models/midfielder_qt.pickle','rb') as f:
    midfielder_qt = pickle.load(f)

midfielder_model = load_model('./models/midfielder_nn.h5')

    
premier_df = pd.read_csv('./datas/NEW_FM_DATA_MERGED/2021-2022/2021-2022-PremierLeague.csv')
premier_df.head(1)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중 A/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,K Ps/90,태클 성공/90,드리블 성공/90,헤더 성공/90,Cr A/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,7,2,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


In [3]:
premier_df.rename(columns={'Cr C/A':'크로스 성공률','FA':'파울 획득','Int/90':'인터셉트/90','공중 A/90':'공중볼시도/90','K Ps/90':'키패스/90','Cr A/90':'크로스 시도/90'},inplace=True)
premier_df = premier_df.astype({'골':np.int64,'도움':np.int64,'파울 획득':np.int64,'태클 성공률':np.int64})
premier_df.head(1)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중볼시도/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,7,2,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


In [4]:
midfielder_df = premier_df[premier_df['포지션']=='미드필더']

In [5]:
# 0골인 선수 보정
midfielder_df['골'] = midfielder_df['골']+1
midfielder_df['경기당 슈팅/90'] = midfielder_df['경기당 슈팅/90']+1

# Shoot Accuracy Weighted Goal
midfielder_df['SAWG'] = (midfielder_df['경기당 유효 슈팅/90']*1.2 / midfielder_df['경기당 슈팅/90'])*midfielder_df['골']

In [6]:
# Pass Weighted Accuracy
midfielder_df['PWA'] = ((midfielder_df['경기당 패스 성공/90'] / midfielder_df['경기당 패스 시도/90'])) + (midfielder_df['경기당 패스 시도/90']*0.8)

In [7]:
# 0도움인 선수 보정
midfielder_df['도움'] = midfielder_df['도움']+0.1

# Assist Per KeyPass
midfielder_df['APK'] = (midfielder_df['도움'] / midfielder_df['키패스/90']) + midfielder_df['키패스/90']

In [8]:
midfielder_df['태클 시도/90'] = midfielder_df['태클 성공/90']/midfielder_df['태클 성공률']
midfielder_df['태클'] = (midfielder_df['태클 성공/90'])+(midfielder_df['태클 시도/90']) - (midfielder_df['태클 성공률']/100)

In [9]:
features ={'APK','SAWG','기회 창출/90','드리블 성공/90','반칙 수/90','인터셉트/90','크로스 성공률','태클','파울 획득'}

In [10]:
midfielder_df.loc[:,features] = midfielder_qt.transform(midfielder_df.loc[:,features])

In [11]:
midfielder_df = midfielder_df.reset_index(drop=True)

In [14]:
independent_val = ('SAWG', '크로스 성공률', '태클', '반칙 수/90', '인터셉트/90', '기회 창출/90', '파울 획득', 'APK')

In [15]:
midfielder_df['예측PPG']=midfielder_model.predict(tf.constant(midfielder_df.loc[:,independent_val].values))

In [16]:
midfielder_df.sort_values(by='예측PPG',ascending=False).head(5)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기,SAWG,PWA,APK,태클 시도/90,태클,예측PPG
119,Andriy Yarmolenko,2021-2022,미드필더,웨스트 햄,잉글랜드 프리미어 리그,6,2.1,0,13,11,...,0.578156,14.90,3.45,1.32,0.908432,25.372423,1.0,0.049048,0.977976,2.004517
124,Hakim Ziyech,2021-2022,미드필더,첼시,잉글랜드 프리미어 리그,6,6.1,0,21,5,...,0.816633,5.60,3.89,2.19,0.929890,32.179399,1.0,0.037444,0.801091,1.962803
91,Bukayo Saka,2021-2022,미드필더,아스날,잉글랜드 프리미어 리그,14,9.1,0,26,4,...,0.999479,4.70,5.58,1.91,0.995656,29.610443,1.0,0.033855,0.680131,1.941054
126,Saúl,2021-2022,미드필더,첼시,잉글랜드 프리미어 리그,10,14.1,21/7/5 - 22/6/30,31,6,...,0.260521,5.15,0.67,2.11,0.974484,42.347465,1.0,0.026835,0.426220,1.910014
44,Jack Grealish,2021-2022,미드필더,맨시티,잉글랜드 프리미어 리그,6,3.1,0,10,3,...,0.493988,2.16,2.57,2.06,0.743477,40.305994,1.0,0.033800,0.772305,1.887521


In [17]:
midfielder_min = np.min(midfielder_df['예측PPG'].tolist())
midfielder_min

1.3134093284606934

In [18]:
midfielder_df['WAR'] = midfielder_df['예측PPG'].map(lambda x : x-midfielder_min+1)

In [19]:
midfielder_war = midfielder_df.sort_values(by='WAR',ascending=False).copy()
midfielder_war = midfielder_war.reset_index(drop=True)
midfielder_war.head(50)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,헤더 성공/90,크로스 시도/90,평균획득승점/경기,SAWG,PWA,APK,태클 시도/90,태클,예측PPG,WAR
0,Andriy Yarmolenko,2021-2022,미드필더,웨스트 햄,잉글랜드 프리미어 리그,6,2.1,0,13,11,...,14.90,3.45,1.32,0.908432,25.372423,1.0,0.049048,0.977976,2.004517,1.691108
1,Hakim Ziyech,2021-2022,미드필더,첼시,잉글랜드 프리미어 리그,6,6.1,0,21,5,...,5.60,3.89,2.19,0.929890,32.179399,1.0,0.037444,0.801091,1.962803,1.649394
2,Bukayo Saka,2021-2022,미드필더,아스날,잉글랜드 프리미어 리그,14,9.1,0,26,4,...,4.70,5.58,1.91,0.995656,29.610443,1.0,0.033855,0.680131,1.941054,1.627645
3,Saúl,2021-2022,미드필더,첼시,잉글랜드 프리미어 리그,10,14.1,21/7/5 - 22/6/30,31,6,...,5.15,0.67,2.11,0.974484,42.347465,1.0,0.026835,0.426220,1.910014,1.596604
4,Jack Grealish,2021-2022,미드필더,맨시티,잉글랜드 프리미어 리그,6,3.1,0,10,3,...,2.16,2.57,2.06,0.743477,40.305994,1.0,0.033800,0.772305,1.887521,1.574112
5,Harvey Barnes,2021-2022,미드필더,레스터,잉글랜드 프리미어 리그,15,7.1,0,22,14,...,4.05,5.62,1.45,0.997919,28.435275,1.0,0.025116,0.397819,1.874733,1.561324
6,James Maddison,2021-2022,미드필더,레스터,잉글랜드 프리미어 리그,10,3.1,0,21,11,...,2.53,1.51,1.57,0.970544,38.178973,1.0,0.027183,0.330097,1.852767,1.539358
7,Tom Davies,2021-2022,미드필더,에버튼,잉글랜드 프리미어 리그,5,3.1,0,11,9,...,4.21,1.17,1.56,0.658165,44.521043,1.0,0.057051,0.996947,1.841491,1.528082
8,Pablo Fornals,2021-2022,미드필더,웨스트 햄,잉글랜드 프리미어 리그,7,4.1,0,17,14,...,5.12,4.97,1.41,0.846362,36.538159,1.0,0.031831,0.547609,1.834847,1.521438
9,Bruno Fernandes,2021-2022,미드필더,맨 레드,잉글랜드 프리미어 리그,12,10.1,0,21,12,...,1.63,4.09,1.60,0.968882,40.222813,1.0,0.020811,0.026215,1.794366,1.480957


In [20]:
midfielder_war.loc[:,features] = midfielder_qt.inverse_transform(midfielder_war.loc[:,features])

In [21]:
midfielder_war[['이름','시즌','골', '도움','WAR']].sort_values(by='WAR',ascending=False).head(40)

,이름,시즌,골,도움,WAR
0,Andriy Yarmolenko,2021-2022,6,2.1,1.691108
1,Hakim Ziyech,2021-2022,6,6.1,1.649394
2,Bukayo Saka,2021-2022,14,9.1,1.627645
3,Saúl,2021-2022,10,14.1,1.596604
4,Jack Grealish,2021-2022,6,3.1,1.574112
5,Harvey Barnes,2021-2022,15,7.1,1.561324
6,James Maddison,2021-2022,10,3.1,1.539358
7,Tom Davies,2021-2022,5,3.1,1.528082
8,Pablo Fornals,2021-2022,7,4.1,1.521438
9,Bruno Fernandes,2021-2022,12,10.1,1.480957
